In [1]:
import sys
import os
import numpy as np
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parent))
np.set_printoptions(threshold=sys.maxsize, suppress=True)

In [2]:
from loader import Loader
from constant import FeatureType, FileType, DataType
from copy import deepcopy
from skimage.measure import compare_nrmse
from keras.models import Model
from keras.optimizers import RMSprop
from keras.layers import Input, LSTM, Dense, RepeatVector
import time
import argparse
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import hashlib
import json

### Train, Validate, Test

In [3]:
def encoder_decoder(partition, args):
    X_train, y_train = partition['train']
    X_val, y_val = partition['val']

    batch_size = args.batch_size
    epochs = args.epochs
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=args.early_stop)

    with tf.device('/GPU:0'):        
        encoder_inputs = Input(shape=(args.x_frames * 24, len(args.features)))

        encoder = LSTM(args.hid_dim, return_state=True)
        encoder_outputs, state_h, state_c = encoder(encoder_inputs)
        encoder_states = [state_h, state_c]

        repeat_vector = RepeatVector(24)
        repeat_vector_outputs = repeat_vector(encoder_outputs)

        decoder_lstm = LSTM(args.hid_dim, return_sequences=True, return_state=True)
        decoder_outputs, _, _ = decoder_lstm(repeat_vector_outputs, initial_state=encoder_states)

        decoder_dense = Dense(1, activation=args.activation)
        decoder_outputs = decoder_dense(decoder_outputs)

        rmsprop = RMSprop(lr=args.lr)

        model = Model(encoder_inputs, decoder_outputs)
        model.compile(optimizer=rmsprop, loss=tf.keras.losses.MeanSquaredError(), metrics='accuracy')

        history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
              batch_size=batch_size, epochs=epochs, callbacks=[callback])
        
        print(model.summary())

    return model, history

In [4]:
def test(partition, model, args):
    X_test, y_test = partition['test']
    
    y_pred = model.predict(X_test)
    y_pred = y_pred.reshape(y_test.shape[0], y_test.shape[1])

    nrmse = compare_nrmse(y_test, y_pred)
    return y_pred, nrmse

In [5]:
def experiment(args):
    loader = Loader(args)
    partition, scaler = loader.get_dataset()
    
    ts = time.time()
    model, history = encoder_decoder(partition, args)
    te = time.time()
    
    print('Took {:2.2f} sec for training the model'.format(te-ts))
    
    y_pred, test_acc = test(partition, model, args)
    
    y_test = scaler.inverse_transform(partition['test'][1])
    y_pred = scaler.inverse_transform(y_pred)
    
    # ======= Add Result to Dictionary ======= #
    result = {}
    result['train_losses'] = history.history['loss']
    result['val_losses'] = history.history['val_loss']
    result['train_accs'] = history.history['accuracy']
    result['val_accs'] = history.history['val_accuracy']
    result['train_acc'] = np.mean(history.history['accuracy'])
    result['val_acc'] = np.mean(history.history['val_accuracy'])
    result['test_nrmse'] = test_acc
    result['y_test'] = np.asarray(y_test).tolist()
    result['y_pred'] = np.asarray(y_pred).tolist()
    
#     plt.plot(np.asarray(partition['test'][1]).tolist())
#     plt.plot(np.asarray(y_pred).tolist())
#     plt.legend()
#     plt.show()
    
    setattr(args, 'features', str(args.features))
    return vars(args), model, result, [y_test, y_pred]

### Manage Experiment

In [6]:
def get_filepath(setting, filetype):
    exp_name = setting['exp_name']
    hash_key = hashlib.sha1(str(setting).encode()).hexdigest()[:6]
    
    file_format = ""
    directory = ""
    if filetype == FileType.MODEL:
        file_format = 'h5'
        directory = 'models'
    elif filetype == FileType.RESULT:
        file_format = 'json'
        directory = 'results'
        
    filename = "%s-%s.%s" % (exp_name, hash_key, file_format)
    filepath = os.path.join(Path(os.getcwd()).parent.parent, directory, filename)
    return filepath
        
def save_exp_model(setting, model):
    filepath = get_filepath(setting, FileType.MODEL)
    model.save(filepath)

def save_exp_result(setting, result):
    filepath = get_filepath(setting, FileType.RESULT)
    
    result.update(setting)
    with open(filepath, 'w') as f:
        json.dump(result, f)
        
def load_exp_result(exp_name):
    dir_path = os.path.join(Path(os.getcwd()).parent.parent, "results")
    filenames = [f for f in listdeir(dir_path) if isfile(join(dir_path, f)) if '.json' in f]
    list_result = []
    for filename in filenames:
        if exp_name in filename:
            with open(join(dir_path, filename), 'r') as infile:
                results = json.load(infile)
                list_result.append(results)
    df = pd.DataFrame(list_result)
    return df

In [7]:
# ====== Random Seed Initialization ====== #
seed = 1234
tf.random.set_seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.exp_name = "solar power prediction using weather features"

# ====== Data Loading ====== #
args.plant = 126
args.spot = 174
args.start_date = "20190820"
args.end_date = "20200809"
args.batch_size = 64
args.x_frames = 3
args.y_frames = 1

# ====== Model Capacity ===== #
args.hid_dim = 256

# ====== Optimizer & Training ====== #
args.optim = 'RMSprop'
args.activation = 'relu'
args.lss = 'MSE'
args.lr = 0.001
args.epochs = 256
args.early_stop = 30
args.evaluation = 'NRMSE'

# ====== Experiment Variable ====== #
name_var1 = 'features'
list_var1 = [FeatureType.SUNSHINE,
                 FeatureType.HUMIDITY,
                 FeatureType.WIND_SPEED,
                 FeatureType.VISIBILITY,
                 FeatureType.GROUND_TEMPERATURE,
                 FeatureType.WIND_DIRECTION,
                 FeatureType.STEAM_PRESSURE,
                 FeatureType.TEMPERATURE,
                 FeatureType.PRECIPITATION,
                 FeatureType.DEW_POINT_TEMPERATURE,
                 FeatureType.ATMOSPHERIC_PRESSURE]

predictions_df = pd.DataFrame()
y_test = None

for i in range(len(list_var1)):
    sub_list = list_var1[:i + 1]

    setattr(args, name_var1, str(sub_list))
    print(args)

    setattr(args, name_var1, sub_list)   
    setting, model, result, expected_values = experiment(deepcopy(args))
    name = "%dth model" % i
    pred = np.concatenate(expected_values[1])
    predictions_df[name] = pred
    y_test = expected_values[0]
    save_exp_model(setting, model)
    save_exp_result(setting, result)
#     print(result)
#     print()
    
predictions_df['y_test'] = np.concatenate(y_test)
predictions_df.to_csv("results_relu.csv")

Namespace(activation='relu', batch_size=64, early_stop=30, end_date='20200809', epochs=256, evaluation='NRMSE', exp_name='solar power prediction using weather features', features="[<FeatureType.SUNSHINE: '일조(hr)'>]", hid_dim=256, lr=0.001, lss='MSE', optim='RMSprop', plant=126, spot=174, start_date='20190820', x_frames=3, y_frames=1)
train start date: 2019-08-20 00:00:00
train end date: 2020-05-09 00:00:00
val start date: 2020-05-10 00:00:00
val end date: 2020-06-22 00:00:00
test start date: 2020-06-23 00:00:00
test end date: 2020-08-05 00:00:00
train start date: 2019-08-20 00:00:00
train end date: 2020-05-09 00:00:00
val start date: 2020-05-10 00:00:00
val end date: 2020-06-22 00:00:00
test start date: 2020-06-23 00:00:00
test end date: 2020-08-05 00:00:00
Epoch 1/256
5/5 [==============================] - 1s 266ms/step - loss: 0.0768 - accuracy: 0.4940 - val_loss: 0.0281 - val_accuracy: 0.3873
Epoch 2/256
5/5 [==============================] - 0s 13ms/step - loss: 0.0440 - accuracy: 

<ipython-input-4-2dfd02194b65>:7: UserWarning: DEPRECATED: skimage.measure.compare_nrmse has been moved to skimage.metrics.normalized_root_mse. It will be removed from skimage.measure in version 0.18.
  nrmse = compare_nrmse(y_test, y_pred)


Namespace(activation='relu', batch_size=64, early_stop=30, end_date='20200809', epochs=256, evaluation='NRMSE', exp_name='solar power prediction using weather features', features="[<FeatureType.SUNSHINE: '일조(hr)'>, <FeatureType.HUMIDITY: '습도(%)'>]", hid_dim=256, lr=0.001, lss='MSE', optim='RMSprop', plant=126, spot=174, start_date='20190820', x_frames=3, y_frames=1)
train start date: 2019-08-20 00:00:00
train end date: 2020-05-09 00:00:00
val start date: 2020-05-10 00:00:00
val end date: 2020-06-22 00:00:00
test start date: 2020-06-23 00:00:00
test end date: 2020-08-05 00:00:00
train start date: 2019-08-20 00:00:00
train end date: 2020-05-09 00:00:00
val start date: 2020-05-10 00:00:00
val end date: 2020-06-22 00:00:00
test start date: 2020-06-23 00:00:00
test end date: 2020-08-05 00:00:00
Epoch 1/256
5/5 [==============================] - 1s 240ms/step - loss: 0.0440 - accuracy: 0.5387 - val_loss: 0.0281 - val_accuracy: 0.3873
Epoch 2/256
5/5 [==============================] - 0s 19ms

<ipython-input-4-2dfd02194b65>:7: UserWarning: DEPRECATED: skimage.measure.compare_nrmse has been moved to skimage.metrics.normalized_root_mse. It will be removed from skimage.measure in version 0.18.
  nrmse = compare_nrmse(y_test, y_pred)


Namespace(activation='relu', batch_size=64, early_stop=30, end_date='20200809', epochs=256, evaluation='NRMSE', exp_name='solar power prediction using weather features', features="[<FeatureType.SUNSHINE: '일조(hr)'>, <FeatureType.HUMIDITY: '습도(%)'>, <FeatureType.WIND_SPEED: '풍속(m/s)'>]", hid_dim=256, lr=0.001, lss='MSE', optim='RMSprop', plant=126, spot=174, start_date='20190820', x_frames=3, y_frames=1)
train start date: 2019-08-20 00:00:00
train end date: 2020-05-09 00:00:00
val start date: 2020-05-10 00:00:00
val end date: 2020-06-22 00:00:00
test start date: 2020-06-23 00:00:00
test end date: 2020-08-05 00:00:00
train start date: 2019-08-20 00:00:00
train end date: 2020-05-09 00:00:00
val start date: 2020-05-10 00:00:00
val end date: 2020-06-22 00:00:00
test start date: 2020-06-23 00:00:00
test end date: 2020-08-05 00:00:00
Epoch 1/256
5/5 [==============================] - 1s 190ms/step - loss: 0.3339 - accuracy: 0.4285 - val_loss: 0.0173 - val_accuracy: 0.3873
Epoch 2/256
5/5 [====

...

Epoch 167/256
5/5 [==============================] - 0s 12ms/step - loss: 0.0060 - accuracy: 0.5388 - val_loss: 0.0117 - val_accuracy: 0.3873
Epoch 168/256
5/5 [==============================] - 0s 11ms/step - loss: 0.0068 - accuracy: 0.5384 - val_loss: 0.0213 - val_accuracy: 0.3873
Epoch 169/256
5/5 [==============================] - 0s 11ms/step - loss: 0.0084 - accuracy: 0.5385 - val_loss: 0.0089 - val_accuracy: 0.3873
Epoch 170/256
5/5 [==============================] - 0s 11ms/step - loss: 0.0099 - accuracy: 0.5377 - val_loss: 0.0153 - val_accuracy: 0.3873
Epoch 171/256
5/5 [==============================] - 0s 10ms/step - loss: 0.0065 - accuracy: 0.5388 - val_loss: 0.0119 - val_accuracy: 0.3873
Epoch 172/256
5/5 [==============================] - 0s 10ms/step - loss: 0.0058 - accuracy: 0.5388 - val_loss: 0.0093 - val_accuracy: 0.3873
Epoch 173/256
5/5 [==============================] - 0s 11ms/step - loss: 0.0072 - accuracy: 0.5388 - val_loss: 0.0249 - val_accuracy: 0.3873
Epoch 

Epoch 225/256
5/5 [==============================] - 0s 10ms/step - loss: 0.0041 - accuracy: 0.5388 - val_loss: 0.0105 - val_accuracy: 0.3873
Epoch 226/256
5/5 [==============================] - 0s 10ms/step - loss: 0.0046 - accuracy: 0.5388 - val_loss: 0.0093 - val_accuracy: 0.3873
Epoch 227/256
5/5 [==============================] - 0s 11ms/step - loss: 0.0040 - accuracy: 0.5388 - val_loss: 0.0136 - val_accuracy: 0.3873
Epoch 228/256
5/5 [==============================] - 0s 11ms/step - loss: 0.0046 - accuracy: 0.5388 - val_loss: 0.0128 - val_accuracy: 0.3873
Epoch 229/256
5/5 [==============================] - 0s 10ms/step - loss: 0.0049 - accuracy: 0.5388 - val_loss: 0.0078 - val_accuracy: 0.3873
Epoch 230/256
5/5 [==============================] - 0s 11ms/step - loss: 0.0047 - accuracy: 0.5388 - val_loss: 0.0172 - val_accuracy: 0.3873
Epoch 231/256
5/5 [==============================] - 0s 11ms/step - loss: 0.0051 - accuracy: 0.5388 - val_loss: 0.0112 - val_accuracy: 0.3873
Epoch 

<ipython-input-4-2dfd02194b65>:7: UserWarning: DEPRECATED: skimage.measure.compare_nrmse has been moved to skimage.metrics.normalized_root_mse. It will be removed from skimage.measure in version 0.18.
  nrmse = compare_nrmse(y_test, y_pred)


Namespace(activation='relu', batch_size=64, early_stop=30, end_date='20200809', epochs=256, evaluation='NRMSE', exp_name='solar power prediction using weather features', features="[<FeatureType.SUNSHINE: '일조(hr)'>, <FeatureType.HUMIDITY: '습도(%)'>, <FeatureType.WIND_SPEED: '풍속(m/s)'>, <FeatureType.VISIBILITY: '시정(10m)'>, <FeatureType.GROUND_TEMPERATURE: '지면온도(°C)'>, <FeatureType.WIND_DIRECTION: '풍향(16방위)'>, <FeatureType.STEAM_PRESSURE: '현지기압(hPa)'>, <FeatureType.TEMPERATURE: '기온(°C)'>, <FeatureType.PRECIPITATION: '강수량(mm)'>, <FeatureType.DEW_POINT_TEMPERATURE: '이슬점온도(°C)'>, <FeatureType.ATMOSPHERIC_PRESSURE: '증기압(hPa)'>]", hid_dim=256, lr=0.001, lss='MSE', optim='RMSprop', plant=126, spot=174, start_date='20190820', x_frames=3, y_frames=1)
train start date: 2019-08-20 00:00:00
train end date: 2020-05-09 00:00:00
val start date: 2020-05-10 00:00:00
val end date: 2020-06-22 00:00:00
test start date: 2020-06-23 00:00:00
test end date: 2020-08-05 00:00:00
train start date: 2019-08-20 00:00:

<ipython-input-4-2dfd02194b65>:7: UserWarning: DEPRECATED: skimage.measure.compare_nrmse has been moved to skimage.metrics.normalized_root_mse. It will be removed from skimage.measure in version 0.18.
  nrmse = compare_nrmse(y_test, y_pred)
